In [ ]:
!pip install -q sklearn
%tensorflow_version 2.x

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import tensorflow.compat.v2.feature_column as fc
import tensorflow as tf

In [ ]:

column_names=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
species=['Setosa','Versicolor','Virginca']
train=pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv",names=column_names,header=0)
test=pd.read_csv("https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv",names=column_names,header=0)


(120, 5)

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [ ]:
train_y=train.pop("Species")
test_y=test.pop("Species")

In [ ]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [ ]:
train.shape

(120, 4)

In [ ]:
#input function

def make_input_fn(features,labels,training=True,batch_size=32):
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  if training:
    dataset=dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)

In [ ]:

feature_columns=[tf.feature_column.numeric_column(key=key) for key in train.keys()]
print(feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
classifier=tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                      hidden_units=[30,10],  #[hidden layer nodes,visible layer nodes]
                                      n_classes=3
                                      )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpuokpijxs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
result=classifier.train(input_fn=lambda:make_input_fn(train,train_y,training=True),steps=5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpuokpijxs/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 10000...
INFO:tensorflow:Saving checkpoints for 10000 into /tmp/tmpuokpijxs/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 10000...
INFO:tensorflow:loss = 0.37836644, step = 10000
INFO:tensorflow:global_step/sec: 588.152
INFO:tensorflow:loss = 0.4882283, step = 10100 (0.173 sec)
INFO:tensorflow:global_step/sec: 872.445
INFO:tensorflow:loss = 0.41778052, step = 10200 (0.117 sec)
INFO:tensorflow:global_step/sec: 816.273
INFO:tensorflow:loss = 0.4662251, step = 10300 (0.122 sec)
INFO:tensorflow:global_step/sec: 812.211
INFO:tensorflow:loss = 0.3983071, step = 10400 (0.123 sec)
IN

In [ ]:
result=classifier.evaluate(input_fn=lambda:make_input_fn(test,test_y,training=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-07-15T12:26:54
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpuokpijxs/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.49362s
INFO:tensorflow:Finished evaluation at 2022-07-15-12:26:55
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.93333334, average_loss = 0.43879274, global_step = 15000, loss = 0.43879274
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /tmp/tmpuokpijxs/model.ckpt-15000


In [ ]:
print(f"accuracy: {result['accuracy']}")

accuracy: 0.9333333373069763


In [ ]:
def test_fn(features,batch_size=256):
  return tf.data.Dataset.from_tensor_slices((dict(features))).batch(batch_size)
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
species=['Setosa','Versicolor','Virginca']


In [ ]:

predict={}
for f in features:
  valid=True
  while valid:
    val=input(f+": ")
    if not val.isdigit():valid=False
  predict[f]=[float(val)]

predictions=classifier.predict(input_fn=lambda:test_fn(predict))
for pred in predictions:
  class_id=pred['class_ids'][0]
  probability=pred['probabilities'][class_id]

  print(f'Prediction is {species[class_id]}  accuracy {100*probability}')

KeyboardInterrupt: ignored

In [ ]:
expected=['Setosa','Versicolor','Virginca']

predict_x={
    'SepalLength':[5.1,5.9,6.9],'SepalWidth':[3.3,3.0,3.1],'PetalLength':[1.7,4.2,5.4],'PetalWidth':[0.5,1.5,2.1]
}

for f in range(len(expected)):
  predict={}
  for k,v in predict_x.items():
    predict[k]=[v[f]]
  print(predict)
  predictions=classifier.predict(input_fn=lambda:test_fn(predict))
  for pred in predictions:
    class_id=pred['class_ids'][0]
    probability=pred['probabilities'][class_id]

    print(f'Prediction is {species[class_id]}  accuracy {100*probability}  {species[class_id]==expected[f]}')

{'SepalLength': [5.1], 'SepalWidth': [3.3], 'PetalLength': [1.7], 'PetalWidth': [0.5]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpuokpijxs/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is Setosa  accuracy 88.14080357551575  True
{'SepalLength': [5.9], 'SepalWidth': [3.0], 'PetalLength': [4.2], 'PetalWidth': [1.5]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpuokpijxs/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is Versicolor  accuracy 51.404935121536255  True
{'SepalLength': [6.9], 'SepalWidth': [3.1], 'PetalLength': [5.4], 'PetalWidth': [2.1]}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorfl